# This notebook is generated by Liucheng on Jul 15, 2018 with python2. 

# Question 2a: Your task is to build a text-based classifier for the `womens-clothes` node. From all the products in the dump, select those belonging to the `womens-clothes` category, try to predict what the following category is.

# (1) Read the json file and clean the data:

In [ ]:
import pandas as pd
!ls

PART2.md                   category_structure.png
Part2_solution-Copy1.ipynb json_test.csv
Part2_solution.ipynb       sample_products.csv
Untitled.ipynb             sample_products.json


In [ ]:
# load the json file into a dataframe
df=pd.read_json('sample_products.json',lines=True,encoding='utf-8') 
print(df.head())
print(df.shape)

           _id                                             fields
0  702461485us  {u'brandedName': [u'Chan Luu Rosette Bandana P...
1  431357984us  {u'brandedName': [u'Charter Club 16" Glass Pea...
2  636341892us  {u'brandedName': [u'Swedish Hasbeens Helena Wo...
3  504434949us  {u'brandedName': [u'Tarte Tartelette Tease Eye...
4  687650992us  {u'brandedName': [u'Calvin Klein Reese Mercury...
(145858, 2)


In [ ]:
new_columns=['id','brandedName', 'categories', 'textDescription']

def separate_fields(i):
    """
    Define the function to separate the column of "fields" for every row
    """
    row=[df.iloc[i,0],df.iloc[i,1][u'brandedName'],df.iloc[i,1][u'categories'],df.iloc[i,1][u'textDescription']]
    df_row=pd.DataFrame(row,index=new_columns).T
    return df_row

# Apply the function to separate the column of "fields" of the whole dataframe
df2=pd.DataFrame(columns=new_columns)  # create an empty dataframe
for i in range(len(df.index)):         # row by row
    df_row=separate_fields(i)
    df2=df2.append(df_row)             
    if i%10000==0:                     
        print i                        # monitor
        
df2.head()

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000


,id,brandedName,categories,textDescription
0,702461485us,[Chan Luu Rosette Bandana Print Neckerchief Sc...,"[clothes-shoes-and-jewelry, women, womens-clot...",[Get ready for your weekend trip with the Chan...
0,431357984us,"[Charter Club 16"" Glass Pearl Cluster Bib Neck...","[clothes-shoes-and-jewelry, women, jewelry, ne...",[An elegant assortment of glass pearls create ...
0,636341892us,[Swedish Hasbeens Helena Women's Shoes],"[clothes-shoes-and-jewelry, women, womens-shoe...",[Achieve on-trend timelessness in the simply s...
0,504434949us,[Tarte Tartelette Tease Eyeshadow Palette],"[clothes-shoes-and-jewelry, women, womens-beau...","[Shadow play with this clay-infused, pocket-pe..."
0,687650992us,[Calvin Klein Reese Mercury Structured Flap Cr...,"[clothes-shoes-and-jewelry, women, handbags, s...",[Put your trendsetting style on display with t...


In [ ]:
# set the column of 'id' as the index
df2=df2.set_index('id')   
df2.head()

,brandedName,categories,textDescription
id,,,
702461485us,[Chan Luu Rosette Bandana Print Neckerchief Sc...,"[clothes-shoes-and-jewelry, women, womens-clot...",[Get ready for your weekend trip with the Chan...
431357984us,"[Charter Club 16"" Glass Pearl Cluster Bib Neck...","[clothes-shoes-and-jewelry, women, jewelry, ne...",[An elegant assortment of glass pearls create ...
636341892us,[Swedish Hasbeens Helena Women's Shoes],"[clothes-shoes-and-jewelry, women, womens-shoe...",[Achieve on-trend timelessness in the simply s...
504434949us,[Tarte Tartelette Tease Eyeshadow Palette],"[clothes-shoes-and-jewelry, women, womens-beau...","[Shadow play with this clay-infused, pocket-pe..."
687650992us,[Calvin Klein Reese Mercury Structured Flap Cr...,"[clothes-shoes-and-jewelry, women, handbags, s...",[Put your trendsetting style on display with t...


In [ ]:
# Figure out the size of the list structures in the columns of 'brandedName', 'categories' and 'textDescription'
size=[]
for i in range(len(df2.index)):
    line_size=(len(df2.iloc[i,0]),len(df2.iloc[i,1]),len(df2.iloc[i,2]))
    if line_size not in size:
        size.append(line_size)
        
size.sort()       
size      # the size of the list-structures in three columns, display duplicate values only once

[(1, 2, 1),
 (1, 3, 1),
 (1, 4, 1),
 (1, 5, 1),
 (1, 6, 1),
 (1, 7, 1),
 (1, 8, 1),
 (1, 9, 1),
 (1, 10, 1),
 (1, 11, 1),
 (1, 12, 1),
 (1, 13, 1),
 (1, 14, 1),
 (1, 15, 1),
 (1, 16, 1),
 (1, 17, 1),
 (1, 18, 1),
 (1, 19, 1),
 (1, 20, 1),
 (1, 21, 1),
 (1, 22, 1),
 (1, 23, 1),
 (1, 24, 1),
 (1, 25, 1),
 (1, 26, 1),
 (1, 28, 1),
 (1, 30, 1),
 (1, 31, 1),
 (1, 35, 1),
 (1, 44, 1)]

In [ ]:
# extract the only string from the lists in the columns of 'brandedName' and 'textDescription'
df2['brandedName']=df2['brandedName'].apply(lambda x:x[0])
df2['textDescription']=df2['textDescription'].apply(lambda x:x[0])

# extract the 3rd and 4th nodes in the categories 
df2['categories_node3']=df2['categories'].apply(lambda x:x[2:3])
df2['categories_node4']=df2['categories'].apply(lambda x:x[3:4])
df2.head()

,brandedName,categories,textDescription,categories_node3,categories_node4
id,,,,,
702461485us,Chan Luu Rosette Bandana Print Neckerchief Sca...,"[clothes-shoes-and-jewelry, women, womens-clot...",Get ready for your weekend trip with the Chan ...,[womens-clothes],[]
431357984us,"Charter Club 16"" Glass Pearl Cluster Bib Necklace","[clothes-shoes-and-jewelry, women, jewelry, ne...",An elegant assortment of glass pearls create a...,[jewelry],[necklaces]
636341892us,Swedish Hasbeens Helena Women's Shoes,"[clothes-shoes-and-jewelry, women, womens-shoe...",Achieve on-trend timelessness in the simply st...,[womens-shoes],[sandals]
504434949us,Tarte Tartelette Tease Eyeshadow Palette,"[clothes-shoes-and-jewelry, women, womens-beau...","Shadow play with this clay-infused, pocket-per...",[womens-beauty],[makeup]
687650992us,Calvin Klein Reese Mercury Structured Flap Cro...,"[clothes-shoes-and-jewelry, women, handbags, s...",Put your trendsetting style on display with th...,[handbags],[shoulder-bags]


In [ ]:
# transform the lists into the strings, which are easier for the coming comparisons
df2['categories_node3']=df2['categories_node3'].apply(lambda x:str(x))
df2['categories_node4']=df2['categories_node4'].apply(lambda x:str(x))
df2.head()

,brandedName,categories,textDescription,categories_node3,categories_node4
id,,,,,
702461485us,Chan Luu Rosette Bandana Print Neckerchief Sca...,"[clothes-shoes-and-jewelry, women, womens-clot...",Get ready for your weekend trip with the Chan ...,[u'womens-clothes'],[]
431357984us,"Charter Club 16"" Glass Pearl Cluster Bib Necklace","[clothes-shoes-and-jewelry, women, jewelry, ne...",An elegant assortment of glass pearls create a...,[u'jewelry'],[u'necklaces']
636341892us,Swedish Hasbeens Helena Women's Shoes,"[clothes-shoes-and-jewelry, women, womens-shoe...",Achieve on-trend timelessness in the simply st...,[u'womens-shoes'],[u'sandals']
504434949us,Tarte Tartelette Tease Eyeshadow Palette,"[clothes-shoes-and-jewelry, women, womens-beau...","Shadow play with this clay-infused, pocket-per...",[u'womens-beauty'],[u'makeup']
687650992us,Calvin Klein Reese Mercury Structured Flap Cro...,"[clothes-shoes-and-jewelry, women, handbags, s...",Put your trendsetting style on display with th...,[u'handbags'],[u'shoulder-bags']


In [ ]:
df2.shape

(145858, 5)

In [ ]:
# only select products belonging to the `womens-clothes` category
df3=df2[df2['categories_node3']=="[u'womens-clothes']"]
df3.shape

(27971, 5)

In [ ]:
df3.head()

,brandedName,categories,textDescription,categories_node3,categories_node4
id,,,,,
702461485us,Chan Luu Rosette Bandana Print Neckerchief Sca...,"[clothes-shoes-and-jewelry, women, womens-clot...",Get ready for your weekend trip with the Chan ...,[u'womens-clothes'],[]
681213395us,Nic+Zoe Morning Haze Top Women's Clothing,"[clothes-shoes-and-jewelry, women, womens-clot...","Casually sophisticated, wearable clothing. Lon...",[u'womens-clothes'],[u'womens-tops']
679071962us,Flynn Skye Celine Slip Dress Women's Dress,"[clothes-shoes-and-jewelry, women, womens-clot...",Nothing shows off your stunning style like thi...,[u'womens-clothes'],[u'dresses']
666765320us,Alo High Waist Dash Leggings Women's Casual Pants,"[clothes-shoes-and-jewelry, women, womens-clot...",Keep things simple in these Alo Dash Leggings....,[u'womens-clothes'],[u'womens-pants']
693268634us,"Bar III Striped Tunic Sweater, Created for Macy's","[clothes-shoes-and-jewelry, women, womens-clot...","Styled with slits along the sides, this sleeve...",[u'womens-clothes'],[u'sweaters']


In [ ]:
# clean the string by remove the unicode and square bracket
df3['categories_node3']=df3['categories_node3'].apply(lambda x:x[3:]).apply(lambda x:x[:-2])
df3.head()

/Library/Python/2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,brandedName,categories,textDescription,categories_node3,categories_node4
id,,,,,
702461485us,Chan Luu Rosette Bandana Print Neckerchief Sca...,"[clothes-shoes-and-jewelry, women, womens-clot...",Get ready for your weekend trip with the Chan ...,womens-clothes,[]
681213395us,Nic+Zoe Morning Haze Top Women's Clothing,"[clothes-shoes-and-jewelry, women, womens-clot...","Casually sophisticated, wearable clothing. Lon...",womens-clothes,[u'womens-tops']
679071962us,Flynn Skye Celine Slip Dress Women's Dress,"[clothes-shoes-and-jewelry, women, womens-clot...",Nothing shows off your stunning style like thi...,womens-clothes,[u'dresses']
666765320us,Alo High Waist Dash Leggings Women's Casual Pants,"[clothes-shoes-and-jewelry, women, womens-clot...",Keep things simple in these Alo Dash Leggings....,womens-clothes,[u'womens-pants']
693268634us,"Bar III Striped Tunic Sweater, Created for Macy's","[clothes-shoes-and-jewelry, women, womens-clot...","Styled with slits along the sides, this sleeve...",womens-clothes,[u'sweaters']


In [ ]:
# For products that sit in the leaf node of 'womens-clothes' and don't go any further, they have no categories_node4.
# We collect them in the test dataset. 
# The classifier will classify those products into their correct subcategories.
test=df3[df3['categories_node4']=='[]']

# In the trian dataset, we collect products belonging to the 'womens-clothes' category and having valid subcategroies.
# The valid subcategroies (categories_node4) will be regarded as the training labels.
train=df3[df3['categories_node4']!='[]']
print test.shape
print train.shape
print test.shape[0]+train.shape[0]-df3.shape[0]

(760, 5)
(27211, 5)
0


In [ ]:
# remove useless colummns and duplicates rows:
test=test.drop(['categories','categories_node4'], axis=1)
test=test.drop_duplicates(keep='first')
test.shape

(756, 3)

In [ ]:
test.head()

,brandedName,textDescription,categories_node3
id,,,
702461485us,Chan Luu Rosette Bandana Print Neckerchief Sca...,Get ready for your weekend trip with the Chan ...,womens-clothes
635961011us,Rampage Trendy Plus Size Sophie Ripped Ann Was...,So many fabulous details make these plus size ...,womens-clothes
452634982us,Anne Cole Side-Tie Swim Brief Bottoms Women's ...,Mixing and matching is made easy with Anne Col...,womens-clothes
614211759us,Love Squared Trendy Plus Size Lace Fit & Flare...,This party season deserves a spectacular look!...,womens-clothes
540846991us,Alex Evenings Plus Size Printed Glitter Jacket...,This versatile plus size jacket and shell twin...,womens-clothes


In [ ]:
# unique values for each column
print len(test['brandedName'].unique()),len(test['textDescription'].unique()),len(test['categories_node3'].unique())

748 680 1


In [ ]:
#save the cleaned dataframe into a csv file. So I don't need to read the json file anymore
test.to_csv('test.csv',encoding='utf-8')

In [ ]:
train.head()

,brandedName,categories,textDescription,categories_node3,categories_node4
id,,,,,
681213395us,Nic+Zoe Morning Haze Top Women's Clothing,"[clothes-shoes-and-jewelry, women, womens-clot...","Casually sophisticated, wearable clothing. Lon...",womens-clothes,[u'womens-tops']
679071962us,Flynn Skye Celine Slip Dress Women's Dress,"[clothes-shoes-and-jewelry, women, womens-clot...",Nothing shows off your stunning style like thi...,womens-clothes,[u'dresses']
666765320us,Alo High Waist Dash Leggings Women's Casual Pants,"[clothes-shoes-and-jewelry, women, womens-clot...",Keep things simple in these Alo Dash Leggings....,womens-clothes,[u'womens-pants']
693268634us,"Bar III Striped Tunic Sweater, Created for Macy's","[clothes-shoes-and-jewelry, women, womens-clot...","Styled with slits along the sides, this sleeve...",womens-clothes,[u'sweaters']
703159265us,Eleven by Venus Williams - Aztec Collection Fl...,"[clothes-shoes-and-jewelry, women, womens-clot...",Add this fun number to your collection and bri...,womens-clothes,[u'skirts']


In [ ]:
# clean the string by remove the unicode and square bracket
train['categories_node4']=train['categories_node4'].apply(lambda x:x[3:]).apply(lambda x:x[:-2])

/Library/Python/2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# remove useless colummns and duplicates rows:
train=train.drop(['categories'], axis=1)
train=train.drop_duplicates(keep='first')
train.shape

(23117, 4)

In [ ]:
train.head()

,brandedName,textDescription,categories_node3,categories_node4
id,,,,
681213395us,Nic+Zoe Morning Haze Top Women's Clothing,"Casually sophisticated, wearable clothing. Lon...",womens-clothes,womens-tops
679071962us,Flynn Skye Celine Slip Dress Women's Dress,Nothing shows off your stunning style like thi...,womens-clothes,dresses
666765320us,Alo High Waist Dash Leggings Women's Casual Pants,Keep things simple in these Alo Dash Leggings....,womens-clothes,womens-pants
693268634us,"Bar III Striped Tunic Sweater, Created for Macy's","Styled with slits along the sides, this sleeve...",womens-clothes,sweaters
703159265us,Eleven by Venus Williams - Aztec Collection Fl...,Add this fun number to your collection and bri...,womens-clothes,skirts


In [ ]:
# unique values for each column
print (len(train['brandedName'].unique()),len(train['textDescription'].unique()),
       len(train['categories_node3'].unique()),len(train['categories_node4'].unique()))

(22269, 22682, 1, 20)


In [ ]:
# 20 unique values of the training labels
train['categories_node4'].unique()

array(['womens-tops', 'dresses', 'womens-pants', 'sweaters', 'skirts',
       'plus-sizes', 'jackets', 'jeans', 'swimsuits', 'womens-outerwear',
       'petites', 'womens-athletic-clothes', 'teen-girls-clothes',
       'sweatshirts', 'maternity-clothes', 'bridal', 'womens-suits',
       'womens-intimates', 'shorts', 'clothes-shoes-and-jewelry'],
      dtype=object)

In [ ]:
#save the cleaned dataframe as a csv file. So I don't need to read the json file anymore
train.to_csv('train.csv',encoding='utf-8')

# (2) Bulid a Naive Bayes classifier to classify their subcategories.

In [ ]:
#read the csv file. So I don't need to read the json file anymore
import pandas as pd
train=pd.read_csv('train.csv', index_col='id')
train.head()

,brandedName,textDescription,categories_node3,categories_node4
id,,,,
681213395us,Nic+Zoe Morning Haze Top Women's Clothing,"Casually sophisticated, wearable clothing. Lon...",womens-clothes,womens-tops
679071962us,Flynn Skye Celine Slip Dress Women's Dress,Nothing shows off your stunning style like thi...,womens-clothes,dresses
666765320us,Alo High Waist Dash Leggings Women's Casual Pants,Keep things simple in these Alo Dash Leggings....,womens-clothes,womens-pants
693268634us,"Bar III Striped Tunic Sweater, Created for Macy's","Styled with slits along the sides, this sleeve...",womens-clothes,sweaters
703159265us,Eleven by Venus Williams - Aztec Collection Fl...,Add this fun number to your collection and bri...,womens-clothes,skirts


In [ ]:
# put string information together
train['words']=train['brandedName']+' '+train['textDescription']
train.head()

,brandedName,textDescription,categories_node3,categories_node4,words
id,,,,,
681213395us,Nic+Zoe Morning Haze Top Women's Clothing,"Casually sophisticated, wearable clothing. Lon...",womens-clothes,womens-tops,Nic+Zoe Morning Haze Top Women's Clothing Casu...
679071962us,Flynn Skye Celine Slip Dress Women's Dress,Nothing shows off your stunning style like thi...,womens-clothes,dresses,Flynn Skye Celine Slip Dress Women's Dress Not...
666765320us,Alo High Waist Dash Leggings Women's Casual Pants,Keep things simple in these Alo Dash Leggings....,womens-clothes,womens-pants,Alo High Waist Dash Leggings Women's Casual Pa...
693268634us,"Bar III Striped Tunic Sweater, Created for Macy's","Styled with slits along the sides, this sleeve...",womens-clothes,sweaters,"Bar III Striped Tunic Sweater, Created for Mac..."
703159265us,Eleven by Venus Williams - Aztec Collection Fl...,Add this fun number to your collection and bri...,womens-clothes,skirts,Eleven by Venus Williams - Aztec Collection Fl...


In [ ]:
train.describe()

,brandedName,textDescription,categories_node3,categories_node4,words
count,23117,23117,23117,23117,23117
unique,22269,22682,1,20,23100
top,A Pea in the Pod Maternity Open-Front Cardigan,Keep your feminine edge while still broadcasti...,womens-clothes,plus-sizes,Hanky Panky Valerie Signature Lace Bralette 48...
freq,13,30,23117,2114,2


In [ ]:
# put two columns of the dataframe into the structure of bunch
import numpy as np
from sklearn.datasets.base import Bunch

texts=[]   # initialize an empty list
targets=np.zeros((train.shape[0],),dtype=np.object)
for i in range(train.shape[0]):
    texts.append(train.words[i])
    targets[i]=train.categories_node4[i]
    
dataset=Bunch(data=texts, target=targets)
print type(dataset)

<class 'sklearn.utils.Bunch'>


In [ ]:
# split data into training data and validation data
from sklearn.cross_validation import train_test_split

x_train,x_validation,y_train,y_validation=train_test_split(dataset.data,dataset.target,test_size=0.2,random_state=98)

/Library/Python/2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
# convert the text into a sparse matrix with tf-idf value as each element
from sklearn.feature_extraction.text import TfidfVectorizer

# tokenize, remove punctuation and words in the stop-words dictionary
my_tfidf=TfidfVectorizer(lowercase=True,analyzer='word',stop_words='english')
# convert the text into a sparse matrix
my_tfidf_x_train=my_tfidf.fit_transform(x_train)
my_tfidf_x_validation=my_tfidf.transform(x_validation)

In [ ]:
# bulid a Naive Bayes classifier to classify their subcategories.
from sklearn.naive_bayes import MultinomialNB

my_NB=MultinomialNB()
# train the model
my_NB.fit(my_tfidf_x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
# return the mean accuracy on the given dataset and the labels
print 'For the train dataset: the mean accuracy is {}.'.format(my_NB.score(my_tfidf_x_train,y_train))
print 'For the validation dataset: the mean accuracy is {}.'.format(my_NB.score(my_tfidf_x_validation, y_validation))

For the train dataset: the mean accuracy is 0.764992159195.
For the validation dataset: the mean accuracy is 0.709991349481.


# My answer:

So my_NB is a text-based Naive Bayes classifier that tries to predict the following subcategory. The mean accuracy is 0.709991349481 for the validation dataset.

# In the original dataset, some products only sit in the leaf node of 'womens-clothes' and don't go any further. My classifier will classify those products into their correct subcategories.

In [ ]:
# read the text dataset
test=pd.read_csv('test.csv', index_col='id')
test.head()

,brandedName,textDescription,categories_node3
id,,,
702461485us,Chan Luu Rosette Bandana Print Neckerchief Sca...,Get ready for your weekend trip with the Chan ...,womens-clothes
635961011us,Rampage Trendy Plus Size Sophie Ripped Ann Was...,So many fabulous details make these plus size ...,womens-clothes
452634982us,Anne Cole Side-Tie Swim Brief Bottoms Women's ...,Mixing and matching is made easy with Anne Col...,womens-clothes
614211759us,Love Squared Trendy Plus Size Lace Fit & Flare...,This party season deserves a spectacular look!...,womens-clothes
540846991us,Alex Evenings Plus Size Printed Glitter Jacket...,This versatile plus size jacket and shell twin...,womens-clothes


In [ ]:
test['words']=test['brandedName']+' '+test['textDescription']
test.head()

,brandedName,textDescription,categories_node3,words
id,,,,
702461485us,Chan Luu Rosette Bandana Print Neckerchief Sca...,Get ready for your weekend trip with the Chan ...,womens-clothes,Chan Luu Rosette Bandana Print Neckerchief Sca...
635961011us,Rampage Trendy Plus Size Sophie Ripped Ann Was...,So many fabulous details make these plus size ...,womens-clothes,Rampage Trendy Plus Size Sophie Ripped Ann Was...
452634982us,Anne Cole Side-Tie Swim Brief Bottoms Women's ...,Mixing and matching is made easy with Anne Col...,womens-clothes,Anne Cole Side-Tie Swim Brief Bottoms Women's ...
614211759us,Love Squared Trendy Plus Size Lace Fit & Flare...,This party season deserves a spectacular look!...,womens-clothes,Love Squared Trendy Plus Size Lace Fit & Flare...
540846991us,Alex Evenings Plus Size Printed Glitter Jacket...,This versatile plus size jacket and shell twin...,womens-clothes,Alex Evenings Plus Size Printed Glitter Jacket...


In [ ]:
# the column of 'words' into a structure of bunch
test_texts=[]
for i in range(test.shape[0]):
    test_texts.append(test.words[i])
    
x_test=Bunch(data=test_texts)

In [ ]:
# vectorize the string to a sparse matrix
my_tfidf_x_test=my_tfidf.transform(x_test.data)

In [ ]:
# get my predictions as a new column of the test dataframe
test['prediction']=my_NB.predict(my_tfidf_x_test)
test.head()

,brandedName,textDescription,categories_node3,words,prediction
id,,,,,
702461485us,Chan Luu Rosette Bandana Print Neckerchief Sca...,Get ready for your weekend trip with the Chan ...,womens-clothes,Chan Luu Rosette Bandana Print Neckerchief Sca...,dresses
635961011us,Rampage Trendy Plus Size Sophie Ripped Ann Was...,So many fabulous details make these plus size ...,womens-clothes,Rampage Trendy Plus Size Sophie Ripped Ann Was...,plus-sizes
452634982us,Anne Cole Side-Tie Swim Brief Bottoms Women's ...,Mixing and matching is made easy with Anne Col...,womens-clothes,Anne Cole Side-Tie Swim Brief Bottoms Women's ...,swimsuits
614211759us,Love Squared Trendy Plus Size Lace Fit & Flare...,This party season deserves a spectacular look!...,womens-clothes,Love Squared Trendy Plus Size Lace Fit & Flare...,plus-sizes
540846991us,Alex Evenings Plus Size Printed Glitter Jacket...,This versatile plus size jacket and shell twin...,womens-clothes,Alex Evenings Plus Size Printed Glitter Jacket...,plus-sizes


My Naive Bayes classifier generates the prediction column for the test dataset.

# Question 2b: What are your takeaways? Are there any particular categories that perform poorly? Why do you think that is? What would you do to improve the performance of this classifier?

In [ ]:
from sklearn.metrics import classification_report

# make predictions on the whole validation data
print classification_report(y_validation,my_NB.predict(my_tfidf_x_validation))

                           precision    recall  f1-score   support

                   bridal       0.55      1.00      0.71        22
clothes-shoes-and-jewelry       0.00      0.00      0.00         0
                  dresses       0.91      0.68      0.78       468
                  jackets       0.44      0.79      0.57       136
                    jeans       0.62      0.62      0.62       110
        maternity-clothes       0.97      0.96      0.97       270
                  petites       0.70      0.70      0.70       252
               plus-sizes       0.60      0.55      0.57       484
                   shorts       0.03      1.00      0.06         3
                   skirts       0.01      1.00      0.02         1
                 sweaters       0.92      0.64      0.75       465
              sweatshirts       0.03      1.00      0.06         4
                swimsuits       0.97      0.82      0.89       445
       teen-girls-clothes       0.65      0.87      0.75     

/Library/Python/2.7/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# My answer:

From the f1-score of the above table, we can see that the categories of 'jackets', 'plus-sizes', 'shorts', 'skirts' and 'sweatshirts' perform poorly (f1-score < 0.60).

(1) We don't have enough samples for the categories of 'shorts', 'skirts' and 'sweatshirts'. A better performance can be expected if we have more training samples of these three categories.

(2) Some products belong to more than one category, which actually have impacts on the performace. In this notebook, I just simply truncate the category list to contain only one category. This may interpret that I don't have very good performance on the category of 'plus-sizes'. I expect that some products with the label 'plus-sizes' actually belong to more than one categories.

(3) Navie Bayes is a simple classifier. Sophisticated classifiers like TextCNN can improve the performance.

(4) Instead of using TF-IDF method, the performance might be improve by using "word2vec" to vectorize the text string.